In [14]:
import pandas as pd
import numpy as np
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [15]:
import pandas as pd
movie_data = pd.read_csv(r"D:\Git repo\Content-Base-Movie-Recommendation\data\movies.csv")

In [16]:
movie_data.head()

,id,title,year,duration,MPA,rating,votes,meta_score,description,Movie_Link,...,opening_weekend_gross,gross_worldwide,gross_us_canada,release_date,countries_origin,filming_locations,production_companies,awards_content,genres,languages
0,tt0073195,Jaws,1975,2h 4m,PG,8.1,690K,87.0,When a massive killer shark unleashes chaos on...,https://www.imdb.com/title/tt0073195,...,"$7,061,513","$477,916,625","$267,263,625",1975.0,['United States'],"[""Water Street, Edgartown, Martha's Vineyard, ...","['Zanuck/Brown Productions', 'Universal Pictur...","Won 3 Oscars, 16 wins & 20 nominations total","['Monster Horror', 'Sea Adventure', 'Survival'...",['English']
1,tt0073629,The Rocky Horror Picture Show,1975,1h 40m,R,7.4,174K,65.0,A newly-engaged couple have a breakdown in an ...,https://www.imdb.com/title/tt0073629,...,NaN,"$115,827,018","$112,892,319",1975.0,"['United Kingdom', 'United States']","[""Oakley Court, Windsor Road, Oakley Green, Wi...","['Twentieth Century Fox', 'Michael White Produ...","Awards, 3 wins & 4 nominations total","['B-Horror', 'Dark Comedy', 'Parody', 'Raunchy...",['English']
2,tt0073486,One Flew Over the Cuckoo's Nest,1975,2h 13m,R,8.7,1.1M,84.0,"In the Fall of 1963, a Korean War veteran and ...",https://www.imdb.com/title/tt0073486,...,NaN,"$109,115,366","$108,981,275",1975.0,['United States'],['Oregon State Mental Hospital - 2600 Center S...,"['Fantasy Films', 'N.V. Zvaluw']","Won 5 Oscars, 38 wins & 15 nominations total","['Medical Drama', 'Psychological Drama', 'Drama']",['English']
3,tt0072890,Dog Day Afternoon,1975,2h 5m,R,8.0,281K,86.0,Three amateur robbers plan to hold up a Brookl...,https://www.imdb.com/title/tt0072890,...,NaN,"$50,004,527","$50,000,000",1975.0,['United States'],"['285 Prospect Park West, Brooklyn, New York C...","['Warner Bros.', 'Artists Entertainment Complex']","Won 1 Oscar, 14 wins & 20 nominations total","['Dark Comedy', 'Heist', 'True Crime', 'Biogra...",['English']
4,tt0073692,Shampoo,1975,1h 50m,R,6.4,15K,65.0,"On Election Day, 1968, irresponsible hairdress...",https://www.imdb.com/title/tt0073692,...,NaN,"$49,407,734","$49,407,734",1975.0,['United States'],"[""2270 Bowmont Drive, Beverly Hills, Californi...","['Persky-Bright / Vista', 'Columbia Pictures',...","Won 1 Oscar, 3 wins & 11 nominations total","['Satire', 'Comedy', 'Drama']",['English']


In [17]:
movie_data.shape

(63249, 24)

In [18]:
movie_data.columns


Index(['id', 'title', 'year', 'duration', 'MPA', 'rating', 'votes',
       'meta_score', 'description', 'Movie_Link', 'writers', 'directors',
       'stars', 'budget', 'opening_weekend_gross', 'gross_worldwide',
       'gross_us_canada', 'release_date', 'countries_origin',
       'filming_locations', 'production_companies', 'awards_content', 'genres',
       'languages'],
      dtype='object')

In [19]:
selected_features = ['genres', 'description', 'directors', 'stars']


In [20]:
for feature in selected_features:
    movie_data[feature] = movie_data[feature].fillna('')

In [22]:
combined_features = movie_data['genres'] + ' ' + movie_data['description'] + ' ' + movie_data['directors'] + ' ' + movie_data['stars']

In [23]:
combined_features

0        ['Monster Horror', 'Sea Adventure', 'Survival'...
1        ['B-Horror', 'Dark Comedy', 'Parody', 'Raunchy...
2        ['Medical Drama', 'Psychological Drama', 'Dram...
3        ['Dark Comedy', 'Heist', 'True Crime', 'Biogra...
4        ['Satire', 'Comedy', 'Drama'] On Election Day,...
                               ...                        
63244    ['Slasher Horror', 'Horror'] A guy and four of...
63245    ['Action', 'Crime', 'Thriller'] When a courier...
63246    ['Adventure', 'Comedy', 'Drama', 'Family'] On ...
63247    ['Monster Horror', 'Supernatural Horror', 'Hor...
63248    ['Adventure'] After a man's wife is killed by ...
Length: 63249, dtype: object

In [24]:
from numpy import vectorize


vectorize = TfidfVectorizer()

In [25]:
feature_vectors = vectorize.fit_transform(combined_features)

In [26]:
feature_vectors

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 3399525 stored elements and shape (63249, 157942)>

In [29]:
# Work with a subset of 10K movies initially
subset_data = movie_data.sample(n=10000, random_state=42)

In [30]:
# Use TfidfVectorizer with sparse output
tfidf = TfidfVectorizer(max_features=5000)
sparse_features = tfidf.fit_transform(combined_features)

In [31]:
# Optimized feature processing
from scipy.sparse import lil_matrix
max_features = 5000
batch_size = 500
tfidf = TfidfVectorizer(max_features=max_features)
sparse_features = tfidf.fit_transform(combined_features)

# Memory-efficient batch processing
similarity_matrix = lil_matrix((sparse_features.shape[0], sparse_features.shape[0]))

for i in range(0, sparse_features.shape[0], batch_size):
    batch_end = min(i+batch_size, sparse_features.shape[0])
    batch = sparse_features[i:batch_end].astype('float32')
    similarity_matrix[i:batch_end] = cosine_similarity(batch, sparse_features)
    
# Clean up memory
del batch
similarity_matrix = similarity_matrix.tocsr()

TypeError: sparse array length is ambiguous; use getnnz() or shape[0]